In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_friedman1
from sklearn.metrics import mean_squared_error
import sklearn.model_selection as ms

In [68]:
import numpy as np
np.random.seed(123)
import collections, copy, pickle
from termcolor import colored
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [3]:
from mlxtend.frequent_patterns import apriori

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [4]:
import util.plot
import util.data

In [5]:
reload(util.plot)
reload(util.data);

In [6]:
with open('data/encoder.pkl', 'rb') as f:
    E = pickle.load(f)

In [7]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=10*1000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=1000)

In [8]:
for k in data.columns:
    if data[k].isna().sum() > 0:
        print('rm ', k)
        data.drop(columns=[k], inplace=True)

varsUsed = list(data.columns)
# varsUsed.remove('Unnamed')
varsUsed.remove('position')
varsUsed.remove('random_bool')
len(varsUsed)

rm  orig_destination_distance
rm  comp1_rate
rm  comp1_inv
rm  comp1_rate_percent_diff
rm  comp2_rate
rm  comp2_inv
rm  comp2_rate_percent_diff
rm  comp3_rate
rm  comp3_inv
rm  comp3_rate_percent_diff
rm  comp4_rate
rm  comp4_inv
rm  comp4_rate_percent_diff
rm  comp5_rate
rm  comp5_inv
rm  comp5_rate_percent_diff
rm  comp6_rate
rm  comp6_inv
rm  comp6_rate_percent_diff
rm  comp7_rate
rm  comp7_inv
rm  comp7_rate_percent_diff
rm  comp8_rate
rm  comp8_inv
rm  comp8_rate_percent_diff


48

In [9]:
E.discretizers

{'srch_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'site_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'visitor_location_country_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'prop_country_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'prop_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'srch_destination_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'srch_adults_count': KBinsDiscretizer(encode='ordinal', n_bins=array([5]), strategy='uniform'),
 'srch_children_count': KBinsDiscretizer(encode='ordinal', n_bins=array([3]), strategy='uniform'),
 'srch_room_count': KBinsDiscretizer(encode='ordinal', n_bins=array([2]), strategy='uniform'),
 'position': KBinsDiscretizer(encode='ordinal', n_bins=array([38]), strategy='uniform'),
 'srch_length_of_stay': KBinsDiscretizer(encode='ordinal',

In [10]:
# TODO encode missing data, i.e. the hotels with no clicks

# note that even though the training score is an integer,
# the predicted score is a float. The remainder terms contain information about confidence.

y_labels = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'score']
y = data['score']
x = data[varsUsed].drop(columns=y_labels)
# x.shape, y.shape
# x_train, x_test, y_train, y_test = ms.train_test_split(x, y, test_size=0.3, random_state=42)
# pd.DataFrame.drop()
xy_train, xy_test = ms.train_test_split(data, random_state=42, test_size=0.3)
y = 'score'
y_train = xy_train[y].values
y_test = xy_test[y].values
x_train = xy_train.drop(columns=[y]).values
x_test = xy_test.drop(columns=[y]).values

# y_train = y_train[:,np.newaxis]
# y_test = y_test[:,np.newaxis]

In [31]:
def cross_validation(model_func, x_train, y_train, k=None, results=None, v=0):
    # Train for 5 folds, returing ROC AUC. You can also try 'accuracy' as a scorer
    n_folds = 5
    # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    scoring1 = 'explained_variance' # explained_variance neg_mean_squared_error r2
    scores_1 = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring=scoring1)
    scoring2 = 'r2' # explained_variance neg_mean_squared_error r2
    scores_2 = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring=scoring2)
    
    if results is not None:
        results[k] = (scores_1, scores_2)
    if v:
        print('scores per fold ', scores_acc)
        print('  mean score    ', np.mean(scores_acc))
        print('  standard dev. ', np.std(scores_acc))
    return (scoring1, scoring2)

In [33]:
kwargs = {'random_state': 123}
models = {'GBoost 2': GradientBoostingRegressor(n_estimators=1000, learning_rate=0.001,
                                max_depth=2, loss='ls', **kwargs),
          'GBoost 4': GradientBoostingRegressor(n_estimators=100, learning_rate=0.01,
                                max_depth=4, loss='ls', **kwargs),
          'RForest 2': RandomForestClassifier(n_estimators=1000, max_depth=2, **kwargs),
          'RForest 4': RandomForestClassifier(n_estimators=1000, max_depth=4, **kwargs),
          'Logit': sklearn.linear_model.LogisticRegression(solver='liblinear', multi_class='ovr', **kwargs),
          'SVR': sklearn.svm.SVR(kernel='linear'),
          'KNN 5': sklearn.neighbors.KNeighborsClassifier(n_neighbors=5),
          'Ensemble Bagging': sklearn.ensemble.BaggingClassifier(n_estimators=100, **kwargs),
         }

results = {}
for k,m in models.items():
    print(k)
    scoring1, scoring2 = cross_validation(m, x_train, y_train, k, results)
#     cross_val_score(m, x_train, y_train, cv=n_folds, v=1)
print('\nscoring:', scoring1, scoring2)

GBoost 2
GBoost 4
RForest 2
RForest 4
Logit
SVR
KNN 5
Ensemble Bagging

scoring: explained_variance r2


In [34]:
print('Model & Mean score1 & Std score1 & Mean score2 & Std score2 \\\\ \n\\hline')
best_k = ''
best_mean = 0
for k, (scores_acc, scores_2) in results.items():
    if np.mean(scores_acc) > best_mean:
        best_mean = np.mean(scores_acc)
        best_k = k
    print('%s & %0.4f & %0.4f & %0.4f & %0.4f\\\\' % (k, np.mean(scores_acc), np.std(scores_acc), np.mean(scores_2), np.std(scores_2)))
print('\nbest acc:', best_k, round(best_mean,4))

Model & Mean score1 & Std score1 & Mean score2 & Std score2 \\ 
\hline
GBoost 2 & 0.8648 & 0.0000 & 0.8642 & 0.0009\\
GBoost 4 & 0.8660 & 0.0000 & 0.8655 & 0.0009\\
RForest 2 & 0.2587 & 0.1572 & 0.2399 & 0.1648\\
RForest 4 & 0.9883 & 0.0029 & 0.9882 & 0.0030\\
Logit & 0.5755 & 0.2589 & 0.5671 & 0.2653\\
SVR & 0.4333 & 0.2321 & 0.4190 & 0.2510\\
KNN 5 & 0.0000 & 0.0000 & -0.0317 & 0.0036\\
Ensemble Bagging & 1.0000 & 0.0000 & 1.0000 & 0.0000\\

best acc: Ensemble Bagging 1.0


In [35]:
best_k = ''
best_mean = 0
for k, (_, scores_2) in results.items():
    if np.mean(scores_2) > best_mean:
        best_mean = np.mean(scores_2)
        best_k = k
print('\nbest score 2:', best_k, round(best_mean,4))


best score 2: Ensemble Bagging 1.0


In [39]:
# performance on test data
for k, est in models.items():
    est.fit(x_train, y_train)
    print(k,':', mean_squared_error(y_test, est.predict(x_test)))

GBoost 2 : 0.12870701795983902
GBoost 4 : 0.1275453693298852
RForest 2 : 0.7466666666666667
RForest 4 : 0.02666666666666667
Logit : 0.10333333333333333
SVR : 0.5285970741544384
KNN 5 : 0.9866666666666667
Ensemble Bagging : 0.0


In [42]:
# performance on training data
for k, est in models.items():
    est.fit(x_train, y_train)
    print(k,':', mean_squared_error(y_train, est.predict(x_train)))

GBoost 2 : 0.1295480166794545
GBoost 4 : 0.1283787775930833
RForest 2 : 0.26857142857142857
RForest 4 : 0.008571428571428572
Logit : 0.002857142857142857
SVR : 0.4532242436615893
KNN 5 : 0.9885714285714285
Ensemble Bagging : 0.0


## Grid search to find best params

In [61]:
0.0001, 1e-4

(0.0001, 0.0001)

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py
kwargs = {'random_state': 123}
params = {}
models = {}
models['GBoost'] = GradientBoostingRegressor(criterion='friedman_mse', loss='ls', tol=1e-4, **kwargs)
params['GBoost'] = {'n_estimators': [100,1000], 
                    'learning_rate': [0.1, 0.01, 0.001],
                    'max_depth': [2,3,4]
                   }
models['Random Forest'] = RandomForestClassifier(criterion='gini', n_jobs=2, **kwargs)
params['Random Forest'] = {'n_estimators': [100,1000], 
                            'max_depth': [2,3,4]
                           }
models['Logit'] = sklearn.linear_model.LogisticRegression(solver='sag', # liblinear, multi_class='ovr'
                            multi_class='multinomial', tol=1e-4, n_jobs=2, **kwargs)
params['Logit'] = {'C': [1, ]}
models['SVR'] = sklearn.svm.SVR(gamma='scale', epsilon=0.1 ,tol=1e-3) # gamma = scale: 1 / n_features * X.var()
params['SVR'] = {'kernel': ['rbf','linear'], 'C': [1, 10, 100]}


scores = ['explained_variance'] # explained_variance neg_mean_squared_error r2

for k, model in models.items():
    for score in scores:
        print("\n# %s (score: %s)" % (k, score))
        assert k in params.keys(), 'models and params should have the same keys'
        clf = sklearn.model_selection.GridSearchCV(model, params[k], cv=5, scoring=score)
        clf.fit(x_train, y_train)
        print("Best params (train)")
        print('\t', clf.best_params_)
        y_true, y_pred = y_test, clf.predict(x_test)
        mse = mean_squared_error(y_true, y_pred)
        print(colored('mse: %0.4f' % mse, 'green'))


# GBoost (score: explained_variance)
Best params (train)
	 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
mse: 0.0000

# Random Forest (score: explained_variance)
